In [ ]:
import csv, json, math, os
from osgeo import ogr
from random import uniform
from shapely.geometry import shape, Point, Polygon

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def RandomPointFromBbox(bbox):
    l,b,r,t = bbox
    point = Point(uniform(l,r),uniform(t,b))
    return point.__geo_interface__['coordinates']

def GetPointsFromData(data, minPointCount = 100.0):
    points = []
    pointCount = max(float(minPointCount), len(data["features"]))
    geom = data["features"][0]["geometry"]
    polygon = shape(geom)
    bbox = polygon.bounds
    for i in range(minPointCount):
        points.append(RandomPointFromBbox(bbox))
    return points


In [ ]:
with open("list.json") as f:
    countryList = json.load(f)
    
with open("centroids.geojson") as f:
    centroids = json.load(f)

In [ ]:
for country in countryList:
    match = False
    for feature in centroids["features"]:
        for name in feature["properties"]["names"]:
            if country['name_en'] == name:
                match = True
                break
        if match:
            break
    if not match:
        print "Missing %s" % country["name_en"]

In [ ]:
unhcrData = []
with open("unhcr_popstats_export_time_series_all_data.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        unhcrData.append(row)

In [ ]:
refugees = {}
for row in unhcrData:
    year = row['Year']
    value = row['Value']
    org = row['Origin']
    dst = row['Country / territory of asylum/residence']
    if int(year) >= 1983:
        if row['Population type'] == 'Refugees (incl. refugee-like situations)':
            if org != 'Various/Unknown' and\
                dst != 'Various/Unknown' and\
                org != 'Stateless' and\
                dst != "Stateless":
                if org not in refugees:
                    refugees[org] = {dst: {'max': 0}}
                if dst not in refugees[org]:
                    refugees[org][dst] = {'max': 0}
                refugees[org][dst][year] = value if value != "*" else "0"


In [ ]:
countryNames = []
for org in refugees:
    countryNames.append(org)
    for dst in refugees[org]:
        countryNames.append(dst)
countryNames = list(set(countryNames))

In [ ]:
missing = []
countryName2Iso = {}
for country in countryNames:
    match = False
    for feature in centroids["features"]:
        for name in feature["properties"]["names"]:
            if unicode(country,"utf8").encode('ascii','ignore') == name:
                countryName2Iso[country] = feature["properties"]["iso_alpha-3"]
                match = True
                break
        if match:
            break
    if not match:
        print "Missing %s" % country
        missing.append(country)

In [ ]:
for org in refugees:
    for dst in refugees[org]:
        values = refugees[org][dst]
        for year in range(1984,2017):
            prevValue = int(values[str(year-1)]) if str(year-1) in values else 0
            currValue = int(values[str(year)]) if str(year) in values else 0
            if currValue - prevValue > values['max']:
                values['max'] = currValue - prevValue
            

In [ ]:
randomPoints = {}
for name in countryName2Iso:
    iso = countryName2Iso[name]
    filename = "gadm28_levels/%s/%s_adm0.geojson" % (iso,iso)
    if os.path.exists(filename):
        with open(filename) as f:
            geojson = json.load(f)
            points = GetPointsFromData(geojson, len(countryNames) - 1)            
            randomPoints[iso] = points
    else:
        print "Missing %s" % filename
        l,b,r,t = (66, 23, 108, 42)
        points = []
        for i in range(len(countryNames)):
            point = Point(uniform(l,r),uniform(t,b))
            points.append(point.__geo_interface__['coordinates'])
        randomPoints[iso] = points